
---

# Reproducible Analysis Pipeline for Journal of Phonetics

**Description:** This notebook contains the complete data processing and statistical analysis pipeline for the study. It includes raw data extraction, filtering, and Generalized Additive Mixed Modeling (GAMM) using `mgcv`.
**System Requirements:** Google Colab (Linux environment), Python 3.x, R 4.x.

---

### Cell 1: Setup & Global Configuration

In [ ]:
# ==============================================================================
# Cell 1: Environment Setup & Configuration
# ==============================================================================
# [User Action Required] Set your main project path here.
# All inputs/outputs will be relative to this path.
BASE_DRIVE_PATH = "/content/drive/MyDrive/Rate-F0_Cova"

# --- 1. System Setup ---
!pip install -q praat-parselmouth pandas tqdm joblib

import os
import glob
import numpy as np
import pandas as pd
import parselmouth
from parselmouth.praat import call
from tqdm.notebook import tqdm
from google.colab import drive

# Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- 2. Global Configuration ---
CONFIG = {
    # Input Paths (Raw Data)
    'input_buckeye': os.path.join(BASE_DRIVE_PATH, "data/01_raw/buckeye/"),
    'input_csj_m':   os.path.join(BASE_DRIVE_PATH, "data/01_raw/csj/"),   # Monologue
    'input_csj_d':   os.path.join(BASE_DRIVE_PATH, "data/01_raw/csj_d/"), # Dialogue

    # Output Paths
    'output_file':   os.path.join(BASE_DRIVE_PATH, "data/02_interim/rate_f0_master.csv"),
    'supp_dir':      os.path.join(BASE_DRIVE_PATH, "results/supplement/"),

    # Local Cache (for faster processing on Colab)
    'local_root':    "/content/data",

    # Analysis Parameters
    'pitch_range': (75, 600),
    'vowels_jp': {'a', 'i', 'u', 'e', 'o'},
    'vowels_en': {'aa', 'ae', 'ah', 'ao', 'aw', 'ay', 'eh', 'er', 'ey', 'ih', 'iy', 'ow', 'oy', 'uh', 'uw'},

    # Filtering Thresholds (Strict)
    'dur_range_strict': (0.03, 0.50), # 30ms - 500ms
    'min_valid_frames': 3,

    # Pause/Boundary Labels for Exclusion
    'pause_labels': {
        'sp', 'pau', 'sil', 'noise', 'garbage',
        '<sil>', '<pz>', '<cl>', '(p', '<noise>', '<vocnoise>', '{b_trans}', '{e_trans}'
    }
}

# --- 3. Initialize Directories & Cache ---
for d in [os.path.dirname(CONFIG['output_file']), CONFIG['supp_dir']]:
    os.makedirs(d, exist_ok=True)

def sync_data_to_local():
    """Copy data from Drive to Colab local VM for speed."""
    print("🚀 Synchronizing data to local VM (this may take a minute)...")
    if not os.path.exists(CONFIG['local_root']):
        os.makedirs(CONFIG['local_root'])

    # Helper to copy specific extensions
    def copy_files(src, dst_name, exts):
        dst = os.path.join(CONFIG['local_root'], dst_name)
        if os.path.exists(src):
            if not os.path.exists(dst): os.makedirs(dst)
            for ext in exts:
                # Quiet copy using system cp
                !find "{src}" -name "{ext}" -exec cp -n {{}} "{dst}/" \; 2>/dev/null
        else:
            print(f"⚠️ Source path not found: {src}")

    copy_files(CONFIG['input_buckeye'], "buckeye", ["*.wav", "*.phones"])
    copy_files(CONFIG['input_csj_m'],   "csj",     ["*.wav", "*.TextGrid"])
    copy_files(CONFIG['input_csj_d'],   "csj_d",   ["*.wav", "*.TextGrid"])
    print("✅ Data synchronization complete.")

sync_data_to_local()

---

### Cell 2: Feature Extraction Classes

In [ ]:
# ==============================================================================
# Cell 2: Robust Feature Extractors
# ==============================================================================
class BaseExtractor:
    def __init__(self, root_dir):
        self.root_dir = root_dir

    def analyze_pitch(self, pitch, start, end):
        """Extract F0 max and validate voicing."""
        try:
            if end <= start: return 0, 0, False
            t1 = call(pitch, "Get frame number from time", start)
            t2 = call(pitch, "Get frame number from time", end)
            values = [call(pitch, "Get value in frame", i, "Hertz") for i in range(int(t1), int(t2) + 1)]
            values = [v for v in values if v > 0] # Filter unvoiced

            if not values: return 0, 0, False

            f0_max = max(values)
            has_jump = False
            if len(values) >= 2:
                # Check for octave jumps/errors (heuristic)
                for k in range(len(values)-1):
                    r = values[k] / values[k+1]
                    if r > 1.8 or r < 0.55:
                        has_jump = True; break
            return f0_max, len(values), has_jump
        except:
            return 0, 0, False

    def is_pause(self, label):
        return str(label).lower() in CONFIG['pause_labels']

class BuckeyeExtractor(BaseExtractor):
    def run(self):
        wav_files = sorted(glob.glob(os.path.join(self.root_dir, "**/*.wav"), recursive=True))
        data = []
        print(f"🇺🇸 Processing Buckeye ({len(wav_files)} files)...")

        for wav in tqdm(wav_files):
            try:
                base = os.path.splitext(wav)[0]
                # Fallback label search
                label_path = base + ".phones"
                if not os.path.exists(label_path):
                    cands = glob.glob(base + "*.phon*")
                    if cands: label_path = cands[0]
                    else: continue

                sound = parselmouth.Sound(wav)
                pitch = sound.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=600)

                # Parse labels
                segments = []
                with open(label_path, 'r', errors='ignore') as f:
                    prev_t = 0.0
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) < 3: continue
                        try:
                            end_t = float(parts[0])
                            lbl = parts[2].lower()
                            segments.append({'s': prev_t, 'e': end_t, 'l': lbl})
                            prev_t = end_t
                        except ValueError: continue

                # Extract vowels
                for i, seg in enumerate(segments):
                    if seg['l'] in CONFIG['vowels_en']:
                        f0, n_val, jump = self.analyze_pitch(pitch, seg['s'], seg['e'])

                        # Context
                        prev_l = segments[i-1]['l'] if i > 0 else "START"
                        next_l = segments[i+1]['l'] if i < len(segments)-1 else "END"

                        data.append({
                            'Dataset': 'Buckeye', 'Language': 'English',
                            'FileID': os.path.basename(wav), 'Speaker': os.path.basename(wav)[:3],
                            'Vowel': seg['l'], 'Duration': seg['e'] - seg['s'],
                            'F0_Max': f0, 'Num_Valid': n_val, 'Has_Jump': jump,
                            'PrevSeg': prev_l, 'NextSeg': next_l,
                            'PrevIsPause': self.is_pause(prev_l), 'NextIsPause': self.is_pause(next_l)
                        })
            except Exception: continue
        return pd.DataFrame(data)

class CSJExtractor(BaseExtractor):
    def _get_target_tier_index(self, tg):
        """
        Robustly identify the Phoneme tier in CSJ TextGrids.
        Strategy: Use the 2nd Interval Tier (standard CSJ format).
        Returns: 1-based index or -1 if failed.
        """
        num_tiers = call(tg, "Get number of tiers")
        interval_tiers = []
        for i in range(1, num_tiers + 1):
            if call(tg, "Is interval tier", i):
                interval_tiers.append(i)

        # CSJ standard: Tier 1 = Word/Phrase, Tier 2 = Phoneme (Seg)
        if len(interval_tiers) >= 2:
            return interval_tiers[1]
        return -1

    def _clean_label(self, lbl):
        if not lbl: return ""
        if lbl.startswith('<'): return lbl.lower()
        return lbl.split('+')[0].split('_')[0].lower()

    def run_generic(self, dataset_name, is_dialogue=False):
        path_key = 'input_csj_d' if is_dialogue else 'input_csj_m'
        # Local cache path
        local_dir = os.path.join(CONFIG['local_root'], "csj_d" if is_dialogue else "csj")

        pattern = "**/*-[LR].wav" if is_dialogue else "**/*.wav"
        wav_files = sorted(glob.glob(os.path.join(local_dir, pattern), recursive=True))
        # Filter mono files for monologue
        if not is_dialogue:
            wav_files = [w for w in wav_files if "-L.wav" not in w and "-R.wav" not in w]

        print(f"🇯🇵 Processing {dataset_name} ({len(wav_files)} files)...")
        data = []

        for wav in tqdm(wav_files):
            try:
                base = os.path.basename(wav)
                session_id = base.split('.')[0]
                if is_dialogue: session_id = session_id[:-2] # remove -L/-R

                tg_path = os.path.join(os.path.dirname(wav), session_id + ".TextGrid")
                if not os.path.exists(tg_path): continue

                tg = parselmouth.read(tg_path)
                sound = parselmouth.Sound(wav)
                pitch = sound.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=600)

                tgt_tier = self._get_target_tier_index(tg)
                if tgt_tier == -1: continue

                n_ints = call(tg, "Get number of intervals", tgt_tier)
                for i in range(1, n_ints+1):
                    lbl_raw = call(tg, "Get label of interval", tgt_tier, i)
                    lbl = self._clean_label(lbl_raw)

                    if lbl in CONFIG['vowels_jp']:
                        s = call(tg, "Get start time of interval", tgt_tier, i)
                        e = call(tg, "Get end time of interval", tgt_tier, i)
                        f0, n_val, jump = self.analyze_pitch(pitch, s, e)

                        # Context
                        prev_raw = call(tg, "Get label of interval", tgt_tier, i-1) if i>1 else "START"
                        next_raw = call(tg, "Get label of interval", tgt_tier, i+1) if i<n_ints else "END"

                        row = {
                            'Dataset': dataset_name, 'Language': 'Japanese',
                            'FileID': base, 'Vowel': lbl, 'Duration': e - s,
                            'F0_Max': f0, 'Num_Valid': n_val, 'Has_Jump': jump,
                            'PrevSeg': self._clean_label(prev_raw), 'NextSeg': self._clean_label(next_raw),
                            'PrevIsPause': self.is_pause(prev_raw), 'NextIsPause': self.is_pause(next_raw)
                        }

                        # Identity Logic
                        if is_dialogue:
                            channel = "L" if "-L" in base else "R"
                            row['Session'] = session_id
                            row['Speaker'] = f"{session_id}-{channel}" # Unique per channel
                        else:
                            row['Speaker'] = session_id # Monologue file = Speaker

                        data.append(row)
            except Exception: continue
        return pd.DataFrame(data)

---

### Cell 3: Data Processing & Feature Engineering

In [ ]:
# ==============================================================================
# Cell 3: Data Processing & Dataset Generation
# ==============================================================================
FORCE_REGENERATE = True # Set to False to reload from CSV

if os.path.exists(CONFIG['output_file']) and not FORCE_REGENERATE:
    print(f"📂 Loading existing dataset: {CONFIG['output_file']}")
    df_full = pd.read_csv(CONFIG['output_file'])
else:
    # 1. Run Extractors
    df_buck = BuckeyeExtractor(os.path.join(CONFIG['local_root'], "buckeye")).run()
    df_mono = CSJExtractor(CONFIG['local_root']).run_generic("CSJ_Monologue", is_dialogue=False)
    df_dial = CSJExtractor(CONFIG['local_root']).run_generic("CSJ_Dialogue", is_dialogue=True)

    df_raw = pd.concat([df_buck, df_mono, df_dial], ignore_index=True)
    print(f"📊 Total Raw Tokens: {len(df_raw):,}")

    # 2. Sanity Check
    csj_mono_count = len(df_raw[df_raw['Dataset'] == 'CSJ_Monologue'])
    if csj_mono_count < 10000:
        print(f"⚠️ WARNING: CSJ Monologue count is suspiciously low ({csj_mono_count}). Check Tier logic.")

    # 3. Filtering Flags
    df_raw['Flag_Dur'] = ~df_raw['Duration'].between(CONFIG['dur_range_strict'][0], CONFIG['dur_range_strict'][1])
    df_raw['Flag_Unvoiced'] = df_raw['Num_Valid'] == 0
    df_raw['Flag_Sparse'] = (df_raw['Num_Valid'] > 0) & (df_raw['Num_Valid'] < CONFIG['min_valid_frames'])
    df_raw['Flag_Jump'] = df_raw['Has_Jump']
    df_raw['Exclude'] = df_raw[['Flag_Dur', 'Flag_Unvoiced', 'Flag_Sparse', 'Flag_Jump']].any(axis=1)

    # 4. Save Filtering Statistics
    # (A) Exclusive Filtering
    def calc_exclusive(df, name):
        total = len(df)
        step1 = df[df['Flag_Dur']]
        rem1 = df[~df['Flag_Dur']]
        step2 = rem1[rem1['Flag_Unvoiced']]
        rem2 = rem1[~rem1['Flag_Unvoiced']]
        step3 = rem2[rem2['Flag_Sparse']]
        rem3 = rem2[~rem2['Flag_Sparse']]
        step4 = rem3[rem3['Flag_Jump']]
        kept = rem3[~rem3['Flag_Jump']]
        return {'Dataset': name, 'Total': total, 'Drop_Duration': len(step1), 'Drop_Unvoiced': len(step2),
                'Drop_Sparse': len(step3), 'Drop_Jump': len(step4), 'Final_Kept': len(kept)}

    excl_stats = [calc_exclusive(df_raw[df_raw['Dataset']==ds], ds) for ds in df_raw['Dataset'].unique()]
    pd.DataFrame(excl_stats).to_csv(os.path.join(CONFIG['supp_dir'], "TableS_filtering_summary_exclusive.csv"), index=False)

    # 5. Clean & Engineer Features
    df_full = df_raw[~df_raw['Exclude']].copy()

    # Semitone Conversion (ref 1Hz)
    df_full['F0_ST'] = 12 * np.log2(df_full['F0_Max'] / 1)

    # Winsorization (per speaker)
    def winsor(g): return g.clip(upper=g.quantile(0.995))
    df_full['F0_Max_Winsor'] = df_full.groupby('Speaker')['F0_Max'].transform(winsor)
    df_full['F0_ST_Winsor'] = 12 * np.log2(df_full['F0_Max_Winsor'] / 1)

    # Mundlak Decomposition (Within/Between)
    spk_mean = df_full.groupby('Speaker')['Duration'].transform('mean')
    df_full['Duration_Between'] = spk_mean
    df_full['Duration_Within'] = df_full['Duration'] - spk_mean

    # Fill NAs for R compatibility
    for c in ['Speaker', 'Language', 'Dataset', 'Session', 'NextSeg']:
        if c in df_full.columns: df_full[c] = df_full[c].fillna('NA').astype(str)

    # Save
    df_full.to_csv(CONFIG['output_file'], index=False)
    print(f"✅ Master dataset saved: {len(df_full):,} rows.")

---

### Cell 4: R Environment & Functions

In [ ]:
%load_ext rpy2.ipython

%%R -i df_full
# ==============================================================================
# Cell 4: R Statistical Analysis Setup (mgcv)
# ==============================================================================
library(mgcv)
library(dplyr)

# --- Configuration ---
SUPP_DIR <- "/content/drive/MyDrive/Rate-F0_Cova/results/supplement/"
AUDIT_FILE <- file.path(SUPP_DIR, "TableS_N_audit.csv")
dir.create(SUPP_DIR, showWarnings = FALSE, recursive = TRUE)

# --- Helper 1: N-Audit System ---
# Records strict N counts for every model to ensure reproducibility
record_audit <- function(model_obj, model_id, study_label, input_df) {
  n_input <- nrow(input_df)
  n_used <- nobs(model_obj)
  n_spk <- length(unique(input_df$Speaker))

  # Append to CSV
  row <- data.frame(
    timestamp = Sys.time(), model_id = model_id, study = study_label,
    n_input = n_input, n_used = n_used, n_dropped = n_input - n_used,
    n_speakers = n_spk
  )
  write.table(row, AUDIT_FILE, sep=",", append=file.exists(AUDIT_FILE),
              col.names=!file.exists(AUDIT_FILE), row.names=FALSE)
  cat(sprintf("[Audit] %s (Study %s): N=%d (Used=%d)\n", model_id, study_label, n_input, n_used))
}

# --- Helper 2: Strict Fit Wrapper ---
# Forces correct method/discrete settings and runs audit
fit_bam_strict <- function(formula, data, mod_id, study_lbl, method="fREML", discrete=TRUE) {
  cat(paste("\n🚀 Fitting:", mod_id, "...\n"))
  m <- bam(formula, data=data, method=method, discrete=discrete, nthreads=2)
  record_audit(m, mod_id, study_lbl, data)
  return(m)
}

# --- Data Preparation ---
cat("🛠️ Preparing Analysis Subsets...\n")
data_all <- df_full
# Factor conversion
cols_fac <- c("Speaker", "Vowel", "Language", "Dataset", "Session")
data_all[cols_fac] <- lapply(data_all[cols_fac], as.factor)

# Define Main Subsets
study1_tok <- subset(data_all, Dataset %in% c("CSJ_Monologue", "Buckeye")) %>% droplevels()
study2_tok <- subset(data_all, Dataset == "CSJ_Dialogue") %>% droplevels()

# Create Fixed Datasets (Complete Cases for ML comparison)
s1_cc <- complete.cases(study1_tok[, c("F0_ST", "Duration", "Speaker", "Vowel", "Language")])
study1_ml_dat <- study1_tok[s1_cc, ]

s2_cc <- complete.cases(study2_tok[, c("F0_ST", "Duration", "Speaker", "Vowel", "Session")])
study2_ml_dat <- study2_tok[s2_cc, ]

cat(sprintf("✅ Data Ready: Study 1 (N=%d), Study 2 (N=%d)\n", nrow(study1_ml_dat), nrow(study2_ml_dat)))

---

### Cell 5: Main Analysis Execution

In [ ]:
%%R
# ==============================================================================
# Cell 5: Main Statistical Analysis Execution
# ==============================================================================

# --- Task A: ML Model Comparison (AIC) ---
cat("\n=== Task A: ML Model Comparison ===\n")

# Study 1 Formulas
f1_full <- F0_ST ~ Language + s(Duration, by=Language, k=20) + s(Duration, Speaker, bs="fs", m=1, k=5) + s(Vowel, bs="re")
f1_base <- F0_ST ~ Language + s(Duration, by=Language, k=20) + s(Speaker, bs="re") + s(Vowel, bs="re")

m1_full <- fit_bam_strict(f1_full, study1_ml_dat, "s1_full_ML", "1", method="ML", discrete=FALSE)
m1_base <- fit_bam_strict(f1_base, study1_ml_dat, "s1_base_ML", "1", method="ML", discrete=FALSE)
write.csv(AIC(m1_full, m1_base), file.path(SUPP_DIR, "TableS_ML_comparison_study1.csv"))

# Study 2 Formulas
f2_full <- F0_ST ~ s(Duration, k=20) + s(Duration, Speaker, bs="fs", m=1, k=5) + s(Vowel, bs="re")
f2_base <- F0_ST ~ s(Duration, k=20) + s(Speaker, bs="re") + s(Vowel, bs="re")

m2_full <- fit_bam_strict(f2_full, study2_ml_dat, "s2_full_ML", "2", method="ML", discrete=FALSE)
m2_base <- fit_bam_strict(f2_base, study2_ml_dat, "s2_base_ML", "2", method="ML", discrete=FALSE)
write.csv(AIC(m2_full, m2_base), file.path(SUPP_DIR, "TableS_ML_comparison_study2.csv"))


# --- Task B: k-check Diagnostics ---
cat("\n=== Task B: k-check Diagnostics ===\n")
sink(file.path(SUPP_DIR, "TableS_kcheck_log.txt"))
set.seed(123)
gam.check(m1_full)
gam.check(m2_full)
sink()


# --- Task C: Session Variance Components (Robust) ---
cat("\n=== Task C: Session Variance Components ===\n")
# Formula with Session Random Effect
f_re <- F0_ST ~ s(Duration, k=20) + s(Duration, Speaker, bs="fs", m=1, k=5) + s(Session, bs="re") + s(Vowel, bs="re")
m_re <- fit_bam_strict(f_re, study2_ml_dat, "s2_session_re", "2", method="fREML", discrete=TRUE)

# Robust Extraction
vc_raw <- gam.vcomp(m_re)
if(is.list(vc_raw)) { vc_vec <- unlist(vc_raw); vc_mat <- matrix(vc_vec, ncol=1); rownames(vc_mat) <- names(vc_vec) } else { vc_mat <- as.matrix(vc_raw) }

# Build Summary Table
terms <- rownames(vc_mat)
std_devs <- as.numeric(vc_mat[, 1])
comp_type <- rep("Other", length(terms))
comp_type[grepl("Session", terms)] <- "Session"
comp_type[grepl("scale", terms)] <- "Residual"

df_vc <- data.frame(term=terms, std_dev=std_devs, component=comp_type)
write.csv(df_vc, file.path(SUPP_DIR, "TableS_session_vcomp.csv"), row.names=FALSE)


# --- Task D: Buckeye Segmental Control (Pre-fortis clipping) ---
cat("\n=== Task D: Buckeye Segmental Control ===\n")
# Extract Buckeye & Create Control Variable
buck_dat <- subset(study1_tok, Dataset=="Buckeye")
buck_dat$NextVoiceless <- ifelse(trimws(as.character(buck_dat$NextSeg)) %in% c("p","t","k","f","th","s","sh","ch","hh"), "Voiceless", "Voiced")
buck_dat$NextVoiceless <- as.factor(buck_dat$NextVoiceless)
buck_cc <- buck_dat[complete.cases(buck_dat[, c("F0_ST", "Duration", "NextVoiceless")]), ]

# Compare Models
f_bk_base <- F0_ST ~ s(Duration, k=20) + s(Duration, Speaker, bs="fs", m=1, k=5) + s(Vowel, bs="re")
f_bk_ctrl <- F0_ST ~ NextVoiceless + s(Duration, k=20) + s(Duration, Speaker, bs="fs", m=1, k=5) + s(Vowel, bs="re")

m_bk_base <- fit_bam_strict(f_bk_base, buck_cc, "buck_base", "1_bk", method="ML", discrete=FALSE)
m_bk_ctrl <- fit_bam_strict(f_bk_ctrl, buck_cc, "buck_ctrl", "1_bk", method="ML", discrete=FALSE)

res_bk <- data.frame(
    AIC_Base = AIC(m_bk_base), AIC_Ctrl = AIC(m_bk_ctrl),
    Delta_AIC = AIC(m_bk_ctrl) - AIC(m_bk_base),
    Coef_Voiceless = summary(m_bk_ctrl)$p.table[grep("Voiceless", rownames(summary(m_bk_ctrl)$p.table)), "Estimate"]
)
write.csv(res_bk, file.path(SUPP_DIR, "TableS_buckeye_control.csv"), row.names=FALSE)

cat("\n✅ All analyses completed successfully. Check 'results/supplement/' folder.")